In [ ]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, 3, padding=1) #input -? OUtput? RF
        self.bn1   = nn.BatchNorm2d(8)
        self.pool1 = nn.MaxPool2d(2,2)

        self.conv2 = nn.Conv2d(8, 16, 3, padding=1)
        self.bn2   = nn.BatchNorm2d(16)
        self.pool2 = nn.MaxPool2d(2, 2)
        
        self.conv2a = nn.Conv2d(16, 16, 3, padding=1)
        self.bn2a   = nn.BatchNorm2d(16)
        self.pool2a = nn.MaxPool2d(2, 2)

        self.conv3 = nn.Conv2d(16, 32, 3, padding=1)
        self.bn3   = nn.BatchNorm2d(32)
        self.pool3 = nn.MaxPool2d(2, 2)
        
        self.conv4 = nn.Conv2d(32, 32, 3, padding=1)
        self.bn4   = nn.BatchNorm2d(32)
        self.pool4 = nn.MaxPool2d(2, 2)        
        
        self.conv5 = nn.Conv2d(32, 10, 1, padding=0)
        self.bn5   = nn.BatchNorm2d(10)
        
        self.drop1  = nn.Dropout(p=0.25)

        # self.fc =  nn.Linear(400, 10)

    def forward(self, x):
        x = self.bn2(F.relu(self.conv2(self.pool1(self.bn1(F.relu(self.conv1(x)))))))
        x = self.bn2a(F.relu(self.conv2a(x)))
        x = self.pool3(self.bn4(F.relu(self.conv4(self.drop1(self.pool2(self.bn3(F.relu(self.conv3(x)))))))))
        x = self.bn5(F.relu(self.conv5(x)))
        x = F.adaptive_avg_pool2d(x, output_size=1)
        # print(x.shape, x.size(0))
        # x = x.view(x.size(0), -1)
        # print (x.shape)
        # x = self.fc(x)

        return F.log_softmax(x.squeeze())
      

In [ ]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              80
       BatchNorm2d-2            [-1, 8, 28, 28]              16
         MaxPool2d-3            [-1, 8, 14, 14]               0
            Conv2d-4           [-1, 16, 14, 14]           1,168
       BatchNorm2d-5           [-1, 16, 14, 14]              32
            Conv2d-6           [-1, 16, 14, 14]           2,320
       BatchNorm2d-7           [-1, 16, 14, 14]              32
            Conv2d-8           [-1, 32, 14, 14]           4,640
       BatchNorm2d-9           [-1, 32, 14, 14]              64
        MaxPool2d-10             [-1, 32, 7, 7]               0
          Dropout-11             [-1, 32, 7, 7]               0
           Conv2d-12             [-1, 32, 7, 7]           9,248
      BatchNorm2d-13             [-1, 32, 7, 7]              64
        MaxPool2d-14             [-1, 3

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:42: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [ ]:
model

Net(
  (conv1): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2a): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2a): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool2a): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_

In [ ]:


torch.manual_seed(1)
batch_size = 32

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [ ]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        #print(output.shape)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'epoch={epoch} loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.1f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    return 100. * correct / len(test_loader.dataset)

In [ ]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test_accuracy = test(model, device, test_loader)
    if round(test_accuracy, 1) >= 99.4 :
      print("Test Accuracy of at least 99.4% is achieved. Stopping training!")
      break

epoch=1 loss=0.021707836538553238 batch_id=1874: 100%|██████████| 1875/1875 [00:43<00:00, 43.09it/s]



Test set: Average loss: 0.0475, Accuracy: 9868/10000 (98.7%)



epoch=2 loss=0.0772387832403183 batch_id=1874: 100%|██████████| 1875/1875 [00:43<00:00, 43.07it/s]



Test set: Average loss: 0.0361, Accuracy: 9886/10000 (98.9%)



epoch=3 loss=0.009892283007502556 batch_id=1874: 100%|██████████| 1875/1875 [00:42<00:00, 44.08it/s]


...


Test set: Average loss: 0.0223, Accuracy: 9934/10000 (99.3%)



epoch=9 loss=0.020655212923884392 batch_id=1874: 100%|██████████| 1875/1875 [00:43<00:00, 43.29it/s]



Test set: Average loss: 0.0211, Accuracy: 9933/10000 (99.3%)



epoch=10 loss=0.007523940876126289 batch_id=1874: 100%|██████████| 1875/1875 [00:44<00:00, 42.24it/s]



Test set: Average loss: 0.0202, Accuracy: 9939/10000 (99.4%)

Test Accuracy of at least 99.4% is achieved. Stopping training!
